### P3 Flow Setup

## read CSV

In [21]:
import csv
import cv2
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split


CSV_PATH = "./data/driving_log.csv"

samples = []
with open(CSV_PATH) as csvfile:
    next(csvfile, None)
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)
    
print("total sample count: ", len(samples))

train_validation_samples, test_samples = train_test_split(samples, test_size = 0.2, random_state = 42)

train_samples, validation_samples = train_test_split(train_validation_samples, test_size = 0.2, random_state = 42)

print("train sample count:", len(train_samples), " validation sample count: ", len(validation_samples), "test sample count: ", len(test_samples))

print(train_samples[0])

total sample count:  8036
train sample count: 5142  validation sample count:  1286 test sample count:  1608
['IMG/center_2016_12_01_13_34_44_733.jpg', ' IMG/left_2016_12_01_13_34_44_733.jpg', ' IMG/right_2016_12_01_13_34_44_733.jpg', ' 0.01391724', ' 0.9855326', ' 0', ' 30.18647']


In [22]:
from sklearn.utils import shuffle

IMG_PATH_PREFIX = "./data/IMG/"

def generator(samples, batch_size=64):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = IMG_PATH_PREFIX  + batch_sample[0].split('/')[-1]
                
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])                

                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            
            #X_train = np.expand_dims(X_train, axis=0)
            #y_train = np.expand_dims(y_train, axis=1)
            #print("X_train shape: ", X_train.shape, " y_train shape:", y_train.shape)
            #print("X train: ", X_train)
            yield shuffle(X_train, y_train)


train_generator = generator(train_samples, batch_size = 32)
validation_generator = generator(validation_samples, batch_size = 32)

test_generator = generator(test_samples, batch_size = 32)
            
#print("generator output: \n", next(train_generator))
    

In [23]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers.convolutional import Cropping2D, Convolution2D, MaxPooling2D
from keras.layers.core import Lambda, Dense, Activation, Flatten

image_color_channel = 3
image_height = 160
image_width = 320



model = Sequential()

#cropping layer
crop_top = 50
crop_bottom = 20
crop_left = 2
crop_right = 2
model.add(Cropping2D(cropping=((crop_top,crop_bottom), (crop_left,crop_right)), input_shape=(160,320,3),  dim_ordering='tf'))

#lambda layer: to normalize images to [-0.5, +0.5] 
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))


#conv layer
n_filters = 32
model.add(Convolution2D(n_filters, 3, 3, border_mode = "valid"))

#pooling 
model.add(MaxPooling2D(pool_size=(2,2)))

#drop out
model.add(Dropout(0.5))

#relu
model.add(Activation("relu"))

#conv layer
model.add(Convolution2D(32, 5, 5, border_mode = "valid"))

#pooling 
model.add(MaxPooling2D(pool_size=(2,2)))

#drop out
model.add(Dropout(0.5))

#relu
model.add(Activation("relu"))



model.add(Flatten())
model.add(Dense(1))

model.compile(loss = "mse", optimizer="adam")
print(model.summary())
model.fit_generator(train_generator, samples_per_epoch= len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=3)

score = model.evaluate_generator(test_generator, 1500, max_q_size=10, nb_worker=1, pickle_safe=False)

print('Test score:', score[0])
print('Test accuracy:', score[1])


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
cropping2d_11 (Cropping2D)       (None, 90, 316, 3)    0           cropping2d_input_11[0][0]        
____________________________________________________________________________________________________
lambda_8 (Lambda)                (None, 90, 316, 3)    0           cropping2d_11[0][0]              
____________________________________________________________________________________________________
convolution2d_4 (Convolution2D)  (None, 88, 314, 32)   896         lambda_8[0][0]                   
____________________________________________________________________________________________________
maxpooling2d_3 (MaxPooling2D)    (None, 44, 157, 32)   0           convolution2d_4[0][0]            
___________________________________________________________________________________________

IndexError: invalid index to scalar variable.

In [24]:
print(score)


0.0150724655622


In [25]:
model.save('model.h5')